In [55]:
#importing required Libraries
import pandas as pd   #to create dataframe
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML
import numpy as np  # to count the values (in our case)

In [56]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
#request allow you to send HTTP request
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [57]:
#creating an empty list, so that we can append the values
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
#Note: These three list are added recently, Not explained in the video
description = []
Director = []
Stars = []

In [58]:
#storing the meaningfull required data in the variable
movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

In [59]:
#calling one by one using for loop
for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
    
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    meta  = store.find('div', class_ = 'inline-block ratings-metascore').text.replace('\n', '').replace('Metascore', '') if store.find('div', class_ = 'inline-block ratings-metascore') else '^^^^^^'
    metascore.append(meta)
    #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    #<span class="metascore  favorable">84        </span>
    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)
    
    # Description of the Movies -- Not explained in the Video, But you will figure it out. 
    describe = store.find_all('p', class_ = 'text-muted')
    description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    description.append(description_)
    
    #Cast Details -- Scraping Director name and Stars -- Not explained in Video
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Director.append(cast[0])
    Stars.append([x.strip() for x in cast[1].split(",")])
    
    
#creating a dataframe using pandas librar
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})


In [60]:
#Saving data in Excel file:

movie_DF.to_excel("Top_100_IMDB_Movies.xlsx")

In [61]:
movie_DF.head(7)

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,80,"2,440,713",$28.34M,Two imprisoned men bond over a number of years...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,689,671",$134.97M,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,Soorarai Pottru,2020,153,9.1,^^^^^^,"86,961",*****,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Sudha Kongara,"[Suriya, Madhavan, Paresh Rawal, Aparna Balamu..."
3,The Dark Knight,2008,152,9.0,84,"2,396,782",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
4,The Godfather: Part II,1974,202,9.0,90,"1,173,343",$57.30M,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
5,12 Angry Men,1957,96,9.0,96,"721,672",$4.36M,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
6,Dara iz Jasenovca,2020,130,8.9,^^^^^^,"50,704",*****,Follows the story of a young girl named Dara w...,Predrag Antonijevic,"[Biljana Cekic, Zlatan Vidovic, Anja Stanic, L..."
